<a href="https://colab.research.google.com/github/albalopezdevesa/APRENDIZAJE_AUTOMATICO_23_24/blob/main/Ejercicios_clase/Visualizaci%C3%B3n_de_datos_clase_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
pip install plotly_express

In [2]:
# Importamos librerias
from google.colab import files

import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import pandas as pd
import numpy as np

import statsmodels.api as sm
import math

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Cargar el archivo CSV en un DataFrame
df = pd.read_csv('datos_ejercicio_ventas.csv')

# Mostrar las primeras filas del DataFrame para verificar los datos
print(df.head())

         COUNTRY        SUBBRAND  YEAR  MONTH     SCENARIO FORECAST  \
0       Portugal     Lipton (L3)  2023     12  AI_forecast  AI_P02F   
1  Great Britain     Lipton (L3)  2023     12  AI_forecast  AI_P10F   
2          Spain  Pepsi Max (L3)  2023     12  AI_forecast  AI_P09F   
3  Great Britain        7up (L3)  2024     12  AI_forecast  AI_P10F   
4        Hungary     Lipton (L3)  2023      9  AI_forecast  AI_P03F   

   FORECAST_YEAR         AMOUNT  
0         2023.0  754356.237194  
1         2023.0  560030.558029  
2         2023.0   88501.980847  
3         2023.0  363224.511516  
4         2023.0  396176.120491  


## Limpieza y entendimiento de los datos

Antes de hacer nada, vamos a ver cuántas líneas tenemos de código.

In [4]:
print('Hay', len(df), 'filas en el archivo')

Hay 18666 filas en el archivo


La variable 'principal' es `AMOUNT`, veamos cómo se distribuye, si hay valores negativos y si hay outliers, para quitarlos de nuestro estudio.

In [5]:
# Ver los tipos de cada columna
print(df.dtypes)

COUNTRY           object
SUBBRAND          object
YEAR               int64
MONTH              int64
SCENARIO          object
FORECAST          object
FORECAST_YEAR    float64
AMOUNT           float64
dtype: object


In [6]:
# Verificar que la columna 'AMOUNT' no tenga valores faltantes
df = df.dropna(subset=['AMOUNT'])

In [7]:
# Contar cuántos valores negativos hay en la columna 'AMOUNT'
negativos_count = df[df['AMOUNT'] < 0].shape[0]
print(f"Cantidad de valores negativos en la columna 'AMOUNT': {negativos_count}")

# Eliminar las filas con valores negativos en la columna 'AMOUNT'
df = df[df['AMOUNT'] >= 0]

Cantidad de valores negativos en la columna 'AMOUNT': 10


In [8]:
# Contar cuántos valores negativos hay en la columna 'AMOUNT'
negativos_count = df[df['AMOUNT'] < 0].shape[0]
print(f"Comprobación de valores negativos en la columna 'AMOUNT': {negativos_count}")

Comprobación de valores negativos en la columna 'AMOUNT': 0


Vamos a ver qué países y cuántas ventas hay

In [10]:
print(df['COUNTRY'].value_counts(), "\n")

COUNTRY
Great Britain    4008
Netherlands      2476
Hungary          2378
Denmark          2294
Spain            2152
Norway           2058
Czech            1403
Portugal         1179
Italy             708
Name: count, dtype: int64 



In [11]:
print(df['SUBBRAND'].value_counts(), "\n")

SUBBRAND
Pepsi Max (L3)        5598
Pepsi Regular (L3)    3636
Lipton (L3)           2689
7up (L3)              2688
7up Free (L3)         2343
Mountain Dew (L3)     1702
Name: count, dtype: int64 



Vamos a crear dos bases de datos adicionales, en una de ellas tendremos los 'Actuals' y en la otra las 'AI_Forecast'

In [12]:
# Dividir el DataFrame en dos según la categoría
df_AI_forecast = df[df['SCENARIO'] == 'AI_forecast']
df_Actual = df[df['SCENARIO'] == 'actual']

# Contamos cuántas filas hay en cada df
print('Hay', len(df_AI_forecast), 'predicciones (forecast)')
print('Hay', len(df_Actual), 'gastos actuales (actuals)')

Hay 17766 predicciones (forecast)
Hay 890 gastos actuales (actuals)


**HISTÓRICO ACTUAL**

In [13]:
print('Número de ventas en cada país:')
print(df_Actual['COUNTRY'].value_counts(), "\n")

Número de ventas en cada país:
COUNTRY
Great Britain    120
Norway           114
Hungary          110
Czech            107
Netherlands      100
Spain            100
Portugal          99
Denmark           80
Italy             60
Name: count, dtype: int64 



In [14]:
print('Número de ventas de cada marca:')
print(df_Actual['SUBBRAND'].value_counts(), "\n")

Número de ventas de cada marca:
SUBBRAND
Pepsi Max (L3)        180
Pepsi Regular (L3)    180
7up (L3)              150
7up Free (L3)         147
Lipton (L3)           133
Mountain Dew (L3)     100
Name: count, dtype: int64 



In [15]:
# Paso 1: Crear una columna de fecha combinando 'year' y 'month'
df_Actual['fecha'] = pd.to_datetime(df_Actual[['YEAR', 'MONTH']].assign(day=1))

# Encontrar la venta más antigua
venta_mas_antigua = df_Actual.sort_values(by='fecha').iloc[0]

# Mostrar la venta más antigua
print("Venta más antigua:\n", venta_mas_antigua)

# Entontrar la venta más reciente
venta_mas_reciente = df_Actual.sort_values(by='fecha', ascending=False).iloc[0]

print('\n')

# Mostrar la venta más reciente
print("Venta más reciente:\n", venta_mas_reciente)

Venta más antigua:
 COUNTRY                       Norway
SUBBRAND                    7up (L3)
YEAR                            2023
MONTH                              1
SCENARIO                      actual
FORECAST                         NaN
FORECAST_YEAR                    NaN
AMOUNT                   1485.703014
fecha            2023-01-01 00:00:00
Name: 11273, dtype: object


Venta más reciente:
 COUNTRY                     Portugal
SUBBRAND                    7up (L3)
YEAR                            2024
MONTH                              8
SCENARIO                      actual
FORECAST                         NaN
FORECAST_YEAR                    NaN
AMOUNT                 556479.542507
fecha            2024-08-01 00:00:00
Name: 4959, dtype: object


**HISTÓRICO FORECAST**



In [16]:
print('Número de ventas en cada país:')
print(df_AI_forecast['COUNTRY'].value_counts(), "\n")

Número de ventas en cada país:
COUNTRY
Great Britain    3888
Netherlands      2376
Hungary          2268
Denmark          2214
Spain            2052
Norway           1944
Czech            1296
Portugal         1080
Italy             648
Name: count, dtype: int64 



In [17]:
print('Número de ventas de cada marca:')
print(df_AI_forecast['SUBBRAND'].value_counts(), "\n")

Número de ventas de cada marca:
SUBBRAND
Pepsi Max (L3)        5418
Pepsi Regular (L3)    3456
Lipton (L3)           2556
7up (L3)              2538
7up Free (L3)         2196
Mountain Dew (L3)     1602
Name: count, dtype: int64 



In [18]:
# Paso 1: Crear una columna de fecha combinando 'year' y 'month'
df_AI_forecast['fecha'] = pd.to_datetime(df_AI_forecast[['YEAR', 'MONTH']].assign(day=1))

# Encontrar la venta más antigua
venta_mas_antigua = df_AI_forecast.sort_values(by='fecha').iloc[0]

# Mostrar la venta más antigua
print("Predicción más antigua:\n", venta_mas_antigua)

# Entontrar la venta más reciente
venta_mas_reciente = df_AI_forecast.sort_values(by='fecha', ascending=False).iloc[0]

print('\n')

# Mostrar la venta más reciente
print("Predicción más reciente:\n", venta_mas_reciente)

Predicción más antigua:
 COUNTRY                     Portugal
SUBBRAND                    7up (L3)
YEAR                            2023
MONTH                              1
SCENARIO                 AI_forecast
FORECAST                       AI_PF
FORECAST_YEAR                 2023.0
AMOUNT                 289120.980222
fecha            2023-01-01 00:00:00
Name: 3628, dtype: object


Predicción más reciente:
 COUNTRY                     Portugal
SUBBRAND                 Lipton (L3)
YEAR                            2025
MONTH                              5
SCENARIO                 AI_forecast
FORECAST                     AI_P12F
FORECAST_YEAR                 2023.0
AMOUNT                 697803.914536
fecha            2025-05-01 00:00:00
Name: 11069, dtype: object


**Horizonte de previsión**

In [19]:
sub_df = df_AI_forecast[(df_AI_forecast['COUNTRY'] == 'Portugal') & (df_AI_forecast['SUBBRAND'] == 'Lipton (L3)') & (df_AI_forecast['FORECAST'] == 'AI_P02F') & (df_AI_forecast['FORECAST_YEAR'] == 2023)]

print(len(sub_df))

18


## Preguntas propias del ejercicio

### ¿Cómo se distribuyen las ventas realizadas?
Las ventas realizadas son aquellas que se almacenan en la sub base `df_Actual`

**En cada país**

In [20]:
color_map_country = {
    'Great Britain': '#1f77b4',
    'Portugal': '#ff7f0e',
    'Norway': '#2ca02c',
    'Netherlands' : '#d62728',
    'Hungary' : '#9467bd',
    'Denmark' : '#8c564b',
    'Spain' : '#e377c2',
    'Czech' : '#7f7f7f',
    'Italy' : '#bcbd22'
}

# Crear el diagrama de bigotes para todos los países excepto 'Great Britain'
fig = px.box(df_Actual, x='COUNTRY', y='AMOUNT',
              title="Distribución de Ventas por País",
              labels={'COUNTRY': 'País', 'AMOUNT': 'Cantidad de Venta'},
              color='COUNTRY', points="outliers",
              color_discrete_map = color_map_country)

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar ambos gráficos
fig.show()

In [21]:
# Agrupar las ventas totales por marca
df_grouped = df_Actual.groupby('COUNTRY')['AMOUNT'].sum().reset_index()

# Crear el diagrama de barras
fig = px.bar(df_grouped, x='COUNTRY', y='AMOUNT', title="Total de Ventas por Marca",
             labels={'COUNTRY': 'Pais', 'AMOUNT': 'Cantidad Total de Venta'},
             color='COUNTRY',
             color_discrete_map = color_map_country)  # Opcional: color por marca

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar la figura
fig.show()

In [22]:
# Crear el treemap
fig = px.treemap(df_grouped, path=['COUNTRY'], values='AMOUNT',
                 title="Distribución de Ventas por Marca",
                 labels={'COUNTRY': 'Marca', 'AMOUNT': 'Cantidad Total de Venta'},
                 color='COUNTRY',
                 color_discrete_map = color_map_country)

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar la figura
fig.show()

**En cada mes y año**

In [23]:
# Agrupar los datos por año y mes, calculando la suma de ventas
df_grouped = df_Actual.groupby(['YEAR', 'MONTH'], as_index=False)['AMOUNT'].sum()

# Crear una nueva columna 'DATE' para combinar YEAR y MONTH en un formato de fecha
df_grouped['DATE'] = pd.to_datetime(df_grouped[['YEAR', 'MONTH']].assign(DAY=1))

# Crear el gráfico de líneas sin diferenciar por año
fig = px.line(df_grouped, x='DATE', y='AMOUNT',
              title="Distribución de Ventas por Mes y Año",
              labels={'DATE': 'Fecha', 'AMOUNT': 'Cantidad de Ventas'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico
fig.show()

Vamos a dividirlo por años para ver las diferencias en las ventas entre 2023 y 2024

In [24]:
# Agrupar los datos por año y mes, calculando la suma de ventas (o puedes usar el promedio)
df_grouped = df_Actual.groupby(['YEAR', 'MONTH'], as_index=False)['AMOUNT'].sum()

# Crear el diagrama de líneas
fig = px.line(df_grouped, x='MONTH', y='AMOUNT', color = 'YEAR',
              title="Distribución de Ventas por Mes y Año - Comparación",
              labels={'MONTH': 'Mes', 'AMOUNT': 'Cantidad de Ventas', 'YEAR': 'Año'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico
fig.show()

**En cada Marca**

In [25]:
# Definir la paleta de colores (puedes personalizar estos colores como desees)
color_map_subbrand = {
    'Pepsi Max (L3)': '#d62728',
    'Pepsi Regular (L3)': '#1f77b4',
    'Lipton (L3)': '#FFD94D',
    '7up (L3)' : '#2ca02c',
    '7up Free (L3)' : '#f28930',
    'Mountain Dew (L3)' : '#9467bd'
}

# Crear el diagrama de bigotes para cada país
fig = px.box(df_Actual, x='SUBBRAND', y='AMOUNT', title="Distribución de Ventas por Marca",
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad de Venta'},
             color='SUBBRAND',  # Opcional: color por país para mejorar la visualización
             points="outliers",
             color_discrete_map = color_map_subbrand)  # Muestra los outliers en el diagrama de bigotes

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar la figura
fig.show()

In [26]:
# Agrupar las ventas totales por marca
df_grouped = df_Actual.groupby('SUBBRAND')['AMOUNT'].sum().reset_index()

# Crear el diagrama de barras
fig = px.bar(df_grouped, x='SUBBRAND', y='AMOUNT', title="Total de Ventas por Marca",
             labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad Total de Venta'},
             color='SUBBRAND',
             color_discrete_map = color_map_subbrand)  # Opcional: color por marca

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar la figura
fig.show()

In [27]:
# Crear el treemap
fig = px.treemap(df_grouped, path=['SUBBRAND'], values='AMOUNT',
                 title="Distribución de Ventas por Marca",
                 labels={'SUBBRAND': 'Marca', 'AMOUNT': 'Cantidad Total de Venta'},
                 color='SUBBRAND',  # Aplica color a los nodos
                 color_discrete_map = color_map_subbrand)

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar la figura
fig.show()

### ¿Cuál es la tendencia y estacionalidad de...?
**Todas las ventas del país con menos ventas**

In [28]:
# Agrupar por 'COUNTRY' y calcular la suma total de 'AMOUNT' para cada país
sales_by_country = df_Actual.groupby('COUNTRY')['AMOUNT'].sum()

# Ordenar por la suma de 'AMOUNT' de menor a mayor
sales_by_country_sorted = sales_by_country.sort_values()

print("Ventas por País (de menor a mayor):")
print(sales_by_country_sorted)

# Obtener el país con menos ventas
country_with_least_sales = sales_by_country_sorted.idxmin()

# Mostrar el resultado
print(f"El país con menos ventas es: {country_with_least_sales}")

Ventas por País (de menor a mayor):
COUNTRY
Spain            8.131266e+06
Portugal         3.488807e+07
Czech            3.535164e+07
Hungary          4.157947e+07
Italy            4.345404e+07
Norway           5.143697e+07
Denmark          5.659668e+07
Netherlands      6.395943e+07
Great Britain    3.347786e+08
Name: AMOUNT, dtype: float64
El país con menos ventas es: Spain


In [29]:
# Dividir el DataFrame en dos según la categoría
df_Spain = df_Actual[df_Actual['COUNTRY'] == 'Spain']

# Supongamos que tienes un DataFrame llamado df con columnas 'Date' y 'Value'
# Crear una columna 'Date' combinando 'year' y 'month', y asignando el día 1 para cada mes
df_Spain['Date'] = pd.to_datetime(df_Spain[['YEAR', 'MONTH']].assign(day=1))

# Agrupar por 'Date' y sumar 'AMOUNT' para obtener el monto total por fecha
grouped_df_Spain = df_Spain.groupby('Date')['AMOUNT'].sum().reset_index()

# Crear gráfico de líneas con 'Date' en el eje X y la suma de 'AMOUNT' en el eje Y
fig = px.line(grouped_df_Spain, x='Date', y='AMOUNT',
              title='Ventas en España a lo largo del tiempo',
              markers=True,
              labels={'AMOUNT': 'Monto de Venta', 'Date': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico
fig.show()

In [30]:
# Descomposición de la serie temporal en tendencia, estacionalidad y residuos
result = sm.tsa.seasonal_decompose(grouped_df_Spain['AMOUNT'], model='additive', period=4)

# Crear un gráfico de líneas para la serie original
fig = px.line(grouped_df_Spain, x='Date', y='AMOUNT',
              title='Ventas en España a lo largo del tiempo con Estacionalidad y Tendencia',
              markers=True,
              labels={'AMOUNT': 'Monto de Venta', 'Date': 'Fecha'})

# Agregar las líneas de la tendencia y estacionalidad al gráfico
fig.add_scatter(x=grouped_df_Spain['Date'], y=result.trend, mode='lines', name='Tendencia', line=dict(dash='dash', color='red'))
fig.add_scatter(x=grouped_df_Spain['Date'], y=result.seasonal, mode='lines', name='Estacionalidad', line=dict(dash='dot', color='blue'))

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico con tendencia y estacionalidad
fig.show()


Para un estudio más exhaustivo veremos como es la tendencia y estacionalidad por marcas.

In [31]:
# Agrupar por 'Date' y 'COUNTRY', y sumar 'AMOUNT'
grouped_df_pep = df_Spain.groupby(['Date', 'SUBBRAND'], as_index=False)['AMOUNT'].sum()

# Obtener los países únicos y el número de subplots necesarios
countries = grouped_df_pep['SUBBRAND'].unique()
num_countries = len(countries)

# Configurar el número de columnas y calcular el número de filas necesarias
num_cols = 3
num_rows = math.ceil(num_countries / num_cols)

# Crear el subplot
fig = make_subplots(
    rows=num_rows, cols=num_cols, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=countries
)

# Iterar sobre cada país para agregar la tendencia y la estacionalidad
for i, country in enumerate(countries, 1):
    # Filtrar y ordenar los datos por fecha para el país actual
    country_data = grouped_df_pep[grouped_df_pep['SUBBRAND'] == country].sort_values(by='Date')

    # Descomposición de la serie temporal en tendencia, estacionalidad y residuos
    result = sm.tsa.seasonal_decompose(country_data['AMOUNT'], model='additive', period=4)

    # Calcular la posición de fila y columna para este gráfico
    row = math.ceil(i / num_cols)
    col = i % num_cols if i % num_cols != 0 else num_cols

    # Añadir la serie de ventas
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=country_data['AMOUNT'], mode='lines', name=f'{country} - Ventas', line=dict(color=color_map_subbrand[country])),
        row=row, col=col
    )


# Actualizar el diseño del gráfico
fig.update_layout(
    title='Ventas en España por Marca con Tendencia y Estacionalidad',
    showlegend=True,
    height=300 * num_rows,  # Ajustar la altura de los subgráficos
    yaxis_title='Monto de Venta',
    title_x=0.5  # Centrar el título en la parte superior
)

# Mostrar el gráfico
fig.show()

In [32]:
# Agrupar por 'Date' y 'COUNTRY', y sumar 'AMOUNT'
grouped_df_pep = df_Spain.groupby(['Date', 'SUBBRAND'], as_index=False)['AMOUNT'].sum()

# Obtener los países únicos y el número de subplots necesarios
countries = grouped_df_pep['SUBBRAND'].unique()
num_countries = len(countries)

# Configurar el número de columnas y calcular el número de filas necesarias
num_cols = 3
num_rows = math.ceil(num_countries / num_cols)

# Crear el subplot
fig = make_subplots(
    rows=num_rows, cols=num_cols, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=countries
)

# Iterar sobre cada país para agregar la tendencia y la estacionalidad
for i, country in enumerate(countries, 1):
    # Filtrar y ordenar los datos por fecha para el país actual
    country_data = grouped_df_pep[grouped_df_pep['SUBBRAND'] == country].sort_values(by='Date')

    # Descomposición de la serie temporal en tendencia, estacionalidad y residuos
    result = sm.tsa.seasonal_decompose(country_data['AMOUNT'], model='additive', period=4)

    # Calcular la posición de fila y columna para este gráfico
    row = math.ceil(i / num_cols)
    col = i % num_cols if i % num_cols != 0 else num_cols

    # Añadir la serie de ventas
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=country_data['AMOUNT'], mode='lines', name=f'{country} - Ventas', line=dict(color=color_map_subbrand[country])),
        row=row, col=col)

    # Añadir la línea de tendencia
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=result.trend, mode='lines', name=f'{country} - Tendencia',
                   line=dict(dash='dash', color='red')),
        row=row, col=col
    )

    # Añadir la línea de estacionalidad
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=result.seasonal, mode='lines', name=f'{country} - Estacionalidad',
                   line=dict(dash='dot', color='blue')),
        row=row, col=col
    )

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Ventas en España por Marca con Tendencia y Estacionalidad',
    showlegend=True,
    height=300 * num_rows,  # Ajustar la altura de los subgráficos
    yaxis_title='Monto de Venta',
    title_x=0.5  # Centrar el título en la parte superior
)

# Mostrar el gráfico
fig.show()


**La marca con más ventas**

In [33]:
# Agrupar por 'SUBBRAND' y calcular la suma total de 'AMOUNT' para cada marca
sales_by_subbrand = df_Actual.groupby('SUBBRAND')['AMOUNT'].sum()

# Ordenar por la suma de 'AMOUNT' de mayor a menor
sales_by_subbrand_sorted = sales_by_subbrand.sort_values(ascending=False)

# Obtener la marca con más ventas
subbrand_with_most_sales = sales_by_subbrand_sorted.idxmax()

# Mostrar el resultado
print(f"La marca con más ventas es: {subbrand_with_most_sales}")


La marca con más ventas es: Pepsi Max (L3)


In [34]:
# Dividir el DataFrame en dos según la categoría
df_Pepsi = df_Actual[df_Actual['SUBBRAND'] == 'Pepsi Max (L3)']

# Crear una columna 'Date' combinando 'year' y 'month', y asignando el día 1 para cada mes
df_Pepsi['Date'] = pd.to_datetime(df_Pepsi[['YEAR', 'MONTH']].assign(day=1))

# Agrupar por 'Date' y sumar 'AMOUNT' para obtener el monto total por fecha
grouped_df_Pepsi = df_Pepsi.groupby('Date')['AMOUNT'].sum().reset_index()

# Crear gráfico de líneas con 'Date' en el eje X y la suma de 'AMOUNT' en el eje Y
fig = px.line(grouped_df_Pepsi, x='Date', y='AMOUNT',
              title='Ventas de Pepsi Max a lo largo del tiempo',
              markers=True,
              labels={'AMOUNT': 'Monto de Venta', 'Date': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico
fig.show()

In [35]:
# Descomposición de la serie temporal en tendencia, estacionalidad y residuos
result = sm.tsa.seasonal_decompose(grouped_df_Pepsi['AMOUNT'], model='additive', period=4)

# Crear un gráfico de líneas para la serie original
fig = px.line(grouped_df_Pepsi, x='Date', y='AMOUNT',
              title='Ventas de Pepsi Max a lo largo del tiempo con Estacionalidad y Tendencia',
              markers=True,
              labels={'AMOUNT': 'Monto de Venta', 'Date': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Agregar las líneas de la tendencia y estacionalidad al gráfico
fig.add_scatter(x=grouped_df_Pepsi['Date'], y=result.trend, mode='lines', name='Tendencia', line=dict(dash='dash', color='red'))
fig.add_scatter(x=grouped_df_Pepsi['Date'], y=result.seasonal, mode='lines', name='Estacionalidad', line=dict(dash='dot', color='blue'))

# Mostrar el gráfico con tendencia y estacionalidad
fig.show()

Para un estudio más exhaustivo veremos como es la tendencia y estacionalidad por país.

In [36]:
# Agrupar por 'Date' y 'COUNTRY', y sumar 'AMOUNT'
grouped_df_pep = df_Pepsi.groupby(['Date', 'COUNTRY'], as_index=False)['AMOUNT'].sum()

# Obtener los países únicos y el número de subplots necesarios
countries = grouped_df_pep['COUNTRY'].unique()
num_countries = len(countries)

# Configurar el número de columnas y calcular el número de filas necesarias
num_cols = 3
num_rows = math.ceil(num_countries / num_cols)

# Crear el subplot
fig = make_subplots(
    rows=num_rows, cols=num_cols, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=countries
)

# Iterar sobre cada país para agregar la tendencia y la estacionalidad
for i, country in enumerate(countries, 1):
    # Filtrar y ordenar los datos por fecha para el país actual
    country_data = grouped_df_pep[grouped_df_pep['COUNTRY'] == country].sort_values(by='Date')

    # Descomposición de la serie temporal en tendencia, estacionalidad y residuos
    result = sm.tsa.seasonal_decompose(country_data['AMOUNT'], model='additive', period=4)

    # Calcular la posición de fila y columna para este gráfico
    row = math.ceil(i / num_cols)
    col = i % num_cols if i % num_cols != 0 else num_cols

    # Añadir la serie de ventas
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=country_data['AMOUNT'], mode='lines', name=f'{country} - Ventas', line=dict(color=color_map_country[country])),
        row=row, col=col
    )

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Ventas de Pepsi Max por País con Tendencia y Estacionalidad',
    showlegend=True,
    height=300 * num_rows,  # Ajustar la altura de los subgráficos
    yaxis_title='Monto de Venta',
    title_x=0.5  # Centrar el título en la parte superior
)

# Mostrar el gráfico
fig.show()

In [37]:
# Agrupar por 'Date' y 'COUNTRY', y sumar 'AMOUNT'
grouped_df_pep = df_Pepsi.groupby(['Date', 'COUNTRY'], as_index=False)['AMOUNT'].sum()

# Obtener los países únicos y el número de subplots necesarios
countries = grouped_df_pep['COUNTRY'].unique()
num_countries = len(countries)

# Configurar el número de columnas y calcular el número de filas necesarias
num_cols = 3
num_rows = math.ceil(num_countries / num_cols)

# Crear el subplot
fig = make_subplots(
    rows=num_rows, cols=num_cols, shared_xaxes=True, vertical_spacing=0.1,
    subplot_titles=countries
)

# Iterar sobre cada país para agregar la tendencia y la estacionalidad
for i, country in enumerate(countries, 1):
    # Filtrar y ordenar los datos por fecha para el país actual
    country_data = grouped_df_pep[grouped_df_pep['COUNTRY'] == country].sort_values(by='Date')

    # Descomposición de la serie temporal en tendencia, estacionalidad y residuos
    result = sm.tsa.seasonal_decompose(country_data['AMOUNT'], model='additive', period=4)

    # Calcular la posición de fila y columna para este gráfico
    row = math.ceil(i / num_cols)
    col = i % num_cols if i % num_cols != 0 else num_cols

    # Añadir la serie de ventas
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=country_data['AMOUNT'], mode='lines', name=f'{country} - Ventas', line=dict(color=color_map_country[country])),
        row=row, col=col
    )

    # Añadir la línea de tendencia
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=result.trend, mode='lines', name=f'{country} - Tendencia',
                   line=dict(dash='dash', color='red')),
        row=row, col=col
    )

    # Añadir la línea de estacionalidad
    fig.add_trace(
        go.Scatter(x=country_data['Date'], y=result.seasonal, mode='lines', name=f'{country} - Estacionalidad',
                   line=dict(dash='dot', color='blue')),
        row=row, col=col
    )

# Actualizar el diseño del gráfico
fig.update_layout(
    title='Ventas de Pepsi Max por País con Tendencia y Estacionalidad',
    showlegend=True,
    height=300 * num_rows,  # Ajustar la altura de los subgráficos
    yaxis_title='Monto de Venta',
    title_x=0.5  # Centrar el título en la parte superior
)

# Mostrar el gráfico
fig.show()

### ¿Cuáles son las predicciones hechas en España? ¿Cómo de buenas son?

In [38]:
# Obtener y mostrar los valores distintos en la columna FORECAST_YEAR
distinct_forecast_years = df_AI_forecast['FORECAST_YEAR'].unique()

print("Valores distintos en FORECAST_YEAR:", distinct_forecast_years)

Valores distintos en FORECAST_YEAR: [2023.]


Sabemos que todas las predicciones se han hecho en 2023 (en diferentes meses)

Vamos a ver cómo son las predicciones hechas dependiendo del mes en el que se hayan hecho.

In [39]:
df_Spain2 = df_AI_forecast[df_AI_forecast['COUNTRY'] == 'Spain']

# Agrupar por 'Date' y 'FORECAST', sumando 'AMOUNT' para obtener el monto total por fecha y forecast
grouped_df_Spain2 = df_Spain2.groupby(['fecha', 'FORECAST'], as_index=False)['AMOUNT'].mean()

# Crear gráfico de líneas con 'Date' en el eje X, 'AMOUNT' en el eje Y, y una línea por cada 'FORECAST'
fig = px.line(grouped_df_Spain2, x='fecha', y='AMOUNT', color='FORECAST',
              title='Predicciones hechas en España a lo largo del tiempo por Forecast',
              markers=True,
              labels={'AMOUNT': 'Monto de Venta', 'Date': 'Fecha', 'FORECAST': 'Forecast'})

# Centrar el título del gráfico
fig.update_layout(
    title_x=0.5  # Centra el título horizontalmente
)

# Mostrar el gráfico
fig.show()

In [40]:
# Obtener los valores únicos de FORECAST y el número de subplots necesarios
forecast_values = grouped_df_Spain2['FORECAST'].unique()
num_forecasts = len(forecast_values)

# Configurar el número de columnas (3) y calcular el número de filas necesarias
num_cols = 3
num_rows = math.ceil(num_forecasts / num_cols)

# Crear el subplot con 3 columnas
fig = make_subplots(
    rows=num_rows, cols=num_cols, shared_xaxes=False, shared_yaxes=False,
    subplot_titles=[f"Forecast {forecast}" for forecast in forecast_values]
)

# Iterar sobre cada FORECAST y añadir la línea correspondiente
for i, forecast in enumerate(forecast_values, 1):
    # Filtrar los datos para cada forecast y ordenar por 'Date'
    forecast_data = grouped_df_Spain2[grouped_df_Spain2['FORECAST'] == forecast].sort_values(by='fecha')

    # Calcular la posición de fila y columna para este gráfico
    row = math.ceil(i / num_cols)
    col = i % num_cols if i % num_cols != 0 else num_cols

    # Añadir la línea de 'AMOUNT' en cada subfigura
    fig.add_trace(
        go.Scatter(x=forecast_data['fecha'], y=forecast_data['AMOUNT'], mode='lines+markers', name=f'Forecast {forecast}'),
        row=row, col=col
    )

# Ajustar el diseño del gráfico
fig.update_layout(
    title='Predicciones hechas en España a lo largo del tiempo por Forecast',
    height=300 * num_rows,  # Ajustar la altura según el número de filas
    showlegend=False,
    title_x=0.5  # Centrar el título horizontalmente
)

# Mostrar el gráfico
fig.show()

**COMPARAMOS CON LO OCURRIDO DE VERDAD**


Para ver de la mejor forma posible si las predicciones se ajustan a las ventas reales, lo comprobaremos marca por marca, comparando la linea de ventas reales con la media de las predicciones realizadas.

In [41]:
df_Spain3 = df[df['COUNTRY'] == 'Spain']
df_Spain3['DATE'] = pd.to_datetime(df_Spain3[['YEAR', 'MONTH']].assign(day=1))

In [42]:
import plotly.express as px

# Filtrar el DataFrame para obtener los datos de Pepsi Max
df_Pepsi2 = df_Spain3[df_Spain3['SUBBRAND'] == 'Pepsi Max (L3)']

# Calcular el promedio de las predicciones para cada DATE si el SCENARIO es 'AI'
df_forecast = df_Pepsi2[df_Pepsi2['SCENARIO'] == 'AI_forecast'].groupby('DATE')['AMOUNT'].mean().reset_index()
df_forecast['SCENARIO'] = 'AI_forecast'
df_forecast = df_forecast.rename(columns={'AMOUNT': 'AMOUNT_pred'})  # Renombrar 'AMOUNT' a 'AMOUNT_pred'

# Filtrar los valores reales donde el SCENARIO es 'ACTUAL'
df_real = df_Pepsi2[df_Pepsi2['SCENARIO'] == 'actual'].groupby('DATE')['AMOUNT'].sum().reset_index()
df_real['SCENARIO'] = 'ACTUAL'
df_real = df_real.rename(columns={'AMOUNT': 'AMOUNT_real'})  # Renombrar 'AMOUNT' a 'AMOUNT_real'

# Concatenar las predicciones y los valores reales
df_plot = pd.concat([df_forecast[['DATE', 'AMOUNT_pred', 'SCENARIO']], df_real[['DATE', 'AMOUNT_real', 'SCENARIO']]])

# Crear el gráfico de líneas con Plotly Express
fig = px.line(df_plot, x='DATE', y=['AMOUNT_pred', 'AMOUNT_real'], color='SCENARIO',
              title="Comparación de Ventas de Pepsi Max Reales vs. Predicciones",
              labels={'AMOUNT_pred': 'Predicción', 'AMOUNT_real': 'Valor Real', 'DATE': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico
fig.show()

# Calcular el error absoluto y porcentual
# Unir las predicciones y los valores reales en un solo DataFrame
df_comparado = pd.merge(df_forecast[['DATE', 'AMOUNT_pred']], df_real[['DATE', 'AMOUNT_real']], on='DATE', how='outer')

# Calcular el error absoluto
df_comparado['ERROR_ABS'] = abs(df_comparado['AMOUNT_pred'] - df_comparado['AMOUNT_real'])

# Calcular el error porcentual
df_comparado['ERROR_PCT'] = (df_comparado['ERROR_ABS'] / df_comparado['AMOUNT_real']) * 100

# Calcular el error absoluto promedio y el error porcentual promedio
error_abs_prom = df_comparado['ERROR_ABS'].mean()
error_pct_prom = df_comparado['ERROR_PCT'].mean()

print(f"Error Porcentual Promedio: {error_pct_prom:.2f}%")

Error Porcentual Promedio: 8.49%


In [43]:
df_Pepsi3 =  df_Spain3[df_Spain3['SUBBRAND'] == 'Pepsi Regular (L3)']

# Calcular el promedio de las predicciones para cada DATE si el SCENARIO es 'AI'
df_forecast = df_Pepsi3[df_Pepsi3['SCENARIO'] == 'AI_forecast'].groupby('DATE')['AMOUNT'].mean().reset_index()
df_forecast['SCENARIO'] = 'AI_forecast'
df_forecast = df_forecast.rename(columns={'AMOUNT': 'AMOUNT_pred'})  # Renombrar 'AMOUNT' a 'AMOUNT_pred'

# Filtrar los valores reales donde el SCENARIO es 'ACTUAL'
df_real = df_Pepsi3[df_Pepsi3['SCENARIO'] == 'actual'].groupby('DATE')['AMOUNT'].sum().reset_index()
df_real['SCENARIO'] = 'ACTUAL'
df_real = df_real.rename(columns={'AMOUNT': 'AMOUNT_real'})  # Renombrar 'AMOUNT' a 'AMOUNT_real'

# Concatenar las predicciones y los valores reales
df_plot = pd.concat([df_forecast[['DATE', 'AMOUNT_pred', 'SCENARIO']], df_real[['DATE', 'AMOUNT_real', 'SCENARIO']]])

# Crear el gráfico de líneas con Plotly Express
fig = px.line(df_plot, x='DATE', y=['AMOUNT_pred', 'AMOUNT_real'], color='SCENARIO',
              title="Comparación de Ventas Pepsi Regular Reales vs. Predicciones",
              labels={'AMOUNT_pred': 'Predicción', 'AMOUNT_real': 'Valor Real', 'DATE': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico
fig.show()

# Calcular el error absoluto y porcentual
# Unir las predicciones y los valores reales en un solo DataFrame
df_comparado = pd.merge(df_forecast[['DATE', 'AMOUNT_pred']], df_real[['DATE', 'AMOUNT_real']], on='DATE', how='outer')

# Calcular el error absoluto
df_comparado['ERROR_ABS'] = abs(df_comparado['AMOUNT_pred'] - df_comparado['AMOUNT_real'])

# Calcular el error porcentual
df_comparado['ERROR_PCT'] = (df_comparado['ERROR_ABS'] / df_comparado['AMOUNT_real']) * 100

# Calcular el error absoluto promedio y el error porcentual promedio
error_abs_prom = df_comparado['ERROR_ABS'].mean()
error_pct_prom = df_comparado['ERROR_PCT'].mean()

print(f"Error Porcentual Promedio: {error_pct_prom:.2f}%")

Error Porcentual Promedio: 12.00%


In [44]:
df_Lipton=  df_Spain3[df_Spain3['SUBBRAND'] == 'Lipton (L3)']

# Calcular el promedio de las predicciones para cada DATE si el SCENARIO es 'AI'
df_forecast = df_Lipton[df_Lipton['SCENARIO'] == 'AI_forecast'].groupby('DATE')['AMOUNT'].mean().reset_index()
df_forecast['SCENARIO'] = 'AI_forecast'
df_forecast = df_forecast.rename(columns={'AMOUNT': 'AMOUNT_pred'})  # Renombrar 'AMOUNT' a 'AMOUNT_pred'

# Filtrar los valores reales donde el SCENARIO es 'ACTUAL'
df_real = df_Lipton[df_Lipton['SCENARIO'] == 'actual'].groupby('DATE')['AMOUNT'].sum().reset_index()
df_real['SCENARIO'] = 'ACTUAL'
df_real = df_real.rename(columns={'AMOUNT': 'AMOUNT_real'})  # Renombrar 'AMOUNT' a 'AMOUNT_real'

# Concatenar las predicciones y los valores reales
df_plot = pd.concat([df_forecast[['DATE', 'AMOUNT_pred', 'SCENARIO']], df_real[['DATE', 'AMOUNT_real', 'SCENARIO']]])

# Crear el gráfico de líneas con Plotly Express
fig = px.line(df_plot, x='DATE', y=['AMOUNT_pred', 'AMOUNT_real'], color='SCENARIO',
              title="Comparación de Ventas Lipton Reales vs. Predicciones",
              labels={'AMOUNT_pred': 'Predicción', 'AMOUNT_real': 'Valor Real', 'DATE': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico
fig.show()

# Calcular el error absoluto y porcentual
# Unir las predicciones y los valores reales en un solo DataFrame
df_comparado = pd.merge(df_forecast[['DATE', 'AMOUNT_pred']], df_real[['DATE', 'AMOUNT_real']], on='DATE', how='outer')

# Calcular el error absoluto
df_comparado['ERROR_ABS'] = abs(df_comparado['AMOUNT_pred'] - df_comparado['AMOUNT_real'])

# Calcular el error porcentual
df_comparado['ERROR_PCT'] = (df_comparado['ERROR_ABS'] / df_comparado['AMOUNT_real']) * 100

# Calcular el error absoluto promedio y el error porcentual promedio
error_abs_prom = df_comparado['ERROR_ABS'].mean()
error_pct_prom = df_comparado['ERROR_PCT'].mean()

print(f"Error Porcentual Promedio: {error_pct_prom:.2f}%")

Error Porcentual Promedio: 24.02%


In [45]:
df_up=  df_Spain3[df_Spain3['SUBBRAND'] == '7up (L3)']

# Calcular el promedio de las predicciones para cada DATE si el SCENARIO es 'AI'
df_forecast = df_up[df_up['SCENARIO'] == 'AI_forecast'].groupby('DATE')['AMOUNT'].mean().reset_index()
df_forecast['SCENARIO'] = 'AI_forecast'
df_forecast = df_forecast.rename(columns={'AMOUNT': 'AMOUNT_pred'})  # Renombrar 'AMOUNT' a 'AMOUNT_pred'

# Filtrar los valores reales donde el SCENARIO es 'ACTUAL'
df_real = df_up[df_up['SCENARIO'] == 'actual'].groupby('DATE')['AMOUNT'].sum().reset_index()
df_real['SCENARIO'] = 'ACTUAL'
df_real = df_real.rename(columns={'AMOUNT': 'AMOUNT_real'})  # Renombrar 'AMOUNT' a 'AMOUNT_real'

# Concatenar las predicciones y los valores reales
df_plot = pd.concat([df_forecast[['DATE', 'AMOUNT_pred', 'SCENARIO']], df_real[['DATE', 'AMOUNT_real', 'SCENARIO']]])

# Crear el gráfico de líneas con Plotly Express
fig = px.line(df_plot, x='DATE', y=['AMOUNT_pred', 'AMOUNT_real'], color='SCENARIO',
              title="Comparación de Ventas 7 up Reales vs. Predicciones",
              labels={'AMOUNT_pred': 'Predicción', 'AMOUNT_real': 'Valor Real', 'DATE': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico
fig.show()

# Calcular el error absoluto y porcentual
# Unir las predicciones y los valores reales en un solo DataFrame
df_comparado = pd.merge(df_forecast[['DATE', 'AMOUNT_pred']], df_real[['DATE', 'AMOUNT_real']], on='DATE', how='outer')

# Calcular el error absoluto
df_comparado['ERROR_ABS'] = abs(df_comparado['AMOUNT_pred'] - df_comparado['AMOUNT_real'])

# Calcular el error porcentual
df_comparado['ERROR_PCT'] = (df_comparado['ERROR_ABS'] / df_comparado['AMOUNT_real']) * 100

# Calcular el error absoluto promedio y el error porcentual promedio
error_abs_prom = df_comparado['ERROR_ABS'].mean()
error_pct_prom = df_comparado['ERROR_PCT'].mean()

print(f"Error Porcentual Promedio: {error_pct_prom:.2f}%")

Error Porcentual Promedio: 17.95%


In [46]:
df_upf=  df_Spain3[df_Spain3['SUBBRAND'] == '7up Free (L3)']

# Calcular el promedio de las predicciones para cada DATE si el SCENARIO es 'AI'
df_forecast = df_upf[df_upf['SCENARIO'] == 'AI_forecast'].groupby('DATE')['AMOUNT'].mean().reset_index()
df_forecast['SCENARIO'] = 'AI_forecast'
df_forecast = df_forecast.rename(columns={'AMOUNT': 'AMOUNT_pred'})  # Renombrar 'AMOUNT' a 'AMOUNT_pred'

# Filtrar los valores reales donde el SCENARIO es 'ACTUAL'
df_real = df_upf[df_upf['SCENARIO'] == 'actual'].groupby('DATE')['AMOUNT'].sum().reset_index()
df_real['SCENARIO'] = 'ACTUAL'
df_real = df_real.rename(columns={'AMOUNT': 'AMOUNT_real'})  # Renombrar 'AMOUNT' a 'AMOUNT_real'

# Concatenar las predicciones y los valores reales
df_plot = pd.concat([df_forecast[['DATE', 'AMOUNT_pred', 'SCENARIO']], df_real[['DATE', 'AMOUNT_real', 'SCENARIO']]])

# Crear el gráfico de líneas con Plotly Express
fig = px.line(df_plot, x='DATE', y=['AMOUNT_pred', 'AMOUNT_real'], color='SCENARIO',
              title="Comparación de Ventas 7 up Free Reales vs. Predicciones",
              labels={'AMOUNT_pred': 'Predicción', 'AMOUNT_real': 'Valor Real', 'DATE': 'Fecha'})

# Centrar el título del gráfico
fig.update_layout(title_x=0.5)

# Mostrar el gráfico
fig.show()

# Calcular el error absoluto y porcentual
# Unir las predicciones y los valores reales en un solo DataFrame
df_comparado = pd.merge(df_forecast[['DATE', 'AMOUNT_pred']], df_real[['DATE', 'AMOUNT_real']], on='DATE', how='outer')

# Calcular el error absoluto
df_comparado['ERROR_ABS'] = abs(df_comparado['AMOUNT_pred'] - df_comparado['AMOUNT_real'])

# Calcular el error porcentual
df_comparado['ERROR_PCT'] = (df_comparado['ERROR_ABS'] / df_comparado['AMOUNT_real']) * 100

# Calcular el error absoluto promedio y el error porcentual promedio
error_abs_prom = df_comparado['ERROR_ABS'].mean()
error_pct_prom = df_comparado['ERROR_PCT'].mean()

print(f"Error Porcentual Promedio: {error_pct_prom:.2f}%")

Error Porcentual Promedio: 8.45%
